In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install torch==2.4.0

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_o3hhzpa/unsloth_8d71eafd7a434e89ba81c09ebdf812a6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-_o3hhzpa/unsloth_8d71eafd7a434e89ba81c09ebdf812a6
  Resolved https://github.com/unslothai/unsloth.git to commit d45ade257b8578d63236b389f25e73b5c22bb862
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.2 MB/s eta 0:00:00
   ━━━

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/Qwen2-1.5b-bnb-4bit",
]

# https://huggingface.co/unsloth/Qwen2-1.5B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:

### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    # inputs       = examples.get("input", "")
    outputs      = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
from datasets import load_dataset
dataset = load_dataset("nvidia/HelpSteer", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/35331 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 20,
        max_steps = 120,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/35331 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 35,331 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 120
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,2.281800
2,2.012400
3,2.183800
4,2.179300
5,2.240400
6,2.061200
7,2.174500
8,2.225600
9,2.009500
10,2.152900


In [ ]:
!zip -r /content/qwen2-1.5B-finetune.zip /content/outputs/checkpoint-120

  adding: content/outputs/checkpoint-120/ (stored 0%)
  adding: content/outputs/checkpoint-120/adapter_config.json (deflated 54%)
  adding: content/outputs/checkpoint-120/optimizer.pt (deflated 16%)
  adding: content/outputs/checkpoint-120/adapter_model.safetensors (deflated 8%)
  adding: content/outputs/checkpoint-120/merges.txt (deflated 57%)
  adding: content/outputs/checkpoint-120/tokenizer.json (deflated 72%)
  adding: content/outputs/checkpoint-120/README.md (deflated 66%)
  adding: content/outputs/checkpoint-120/vocab.json (deflated 61%)
  adding: content/outputs/checkpoint-120/added_tokens.json (deflated 39%)
  adding: content/outputs/checkpoint-120/special_tokens_map.json (deflated 58%)
  adding: content/outputs/checkpoint-120/scheduler.pt (deflated 57%)
  adding: content/outputs/checkpoint-120/training_args.bin (deflated 51%)
  adding: content/outputs/checkpoint-120/trainer_state.json (deflated 79%)
  adding: content/outputs/checkpoint-120/rng_state.pth (deflated 25%)
  addin

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("huynd/qwen2-1.5B-HelpSteer")

## Evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from unsloth import FastLanguageModel
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "huynd/qwen2-1.5B-HelpSteer", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
       

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# tokens = tokenizer(**inputs)
# answer = model(tokens)

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
tokens = model.generate(**inputs, tokenizer = tokenizer, max_new_tokens = 64)
print(tokens[0])
answer = tokenizer.decode(tokens[0])
answer

tensor([ 38214,    374,    458,   7600,    429,  16555,    264,   3383,     11,
         34426,    448,    458,   1946,    429,   5707,   4623,   2266,     13,
          9645,    264,   2033,    429,  34901,  44595,    279,   1681,    624,
         14374,  29051,    510,   3838,    374,    264,  11245,  16217,  21271,
           304,  12095,   5267,  14374,   5571,   1447,  14374,   5949,    510,
           785,    468,   3092,    301,  21938,    374,    264,  11245,  16217,
         21271,    304,  12095,     13, 151643], device='cuda:0')


'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nWhat is a famous tall tower in Paris?\n### Input:\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris.<|endoftext|>'

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:

### Response:
"""
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    # inputs       = examples.get("input", "")
    # outputs      = examples["response"]
    texts = []
    for instruction in instructions:
        text = alpaca_prompt.format(instruction)
        texts.append(text)
    return { "text" : texts, }

In [ ]:
from datasets import load_dataset
val_set = load_dataset("nvidia/HelpSteer", split = "validation")
val_set = val_set.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1789 [00:00<?, ? examples/s]

In [ ]:
import re
from tqdm import tqdm

results = []
for example in tqdm(val_set):
    # generate here is a placeholder for your models generations
    inputs = tokenizer([example["text"]], return_tensors = "pt").to("cuda")
    answer = model.generate(**inputs, max_new_tokens=1028)
    answer = tokenizer.decode(answer[0])
    result = answer.split("### Response:\n")[-1]
    # answer = result.group(1)
    result_dict = {
      'question': example["prompt"],
      'answer': result,
      'ground_truth': example["response"]
    }
    results.append(result_dict)

  1%|          | 12/1789 [01:29<3:34:58,  7.26s/it]

In [ ]:
import json
with open('answers.json', 'w') as f:
    json.dump(results, f)